In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://633fc3f91d0f:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.29-cf730c8fc8f6
LOGGING: writing to /hail/large_vcf_analysis/hail-20200319-1207-0.2.29-cf730c8fc8f6.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat

Loading BokehJS ...

In [4]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
# Install a pip package in the current Jupyter kernel :)
import sys
!{sys.executable} -m pip install openpyxl

In [18]:
mt = hl.read_matrix_table('/hail/large_vcf_filtering_annotation/step6_output.mt')

In [19]:
mt.count()

(10742986, 302)

In [20]:
pheno = hl.import_table('/hail/GTS-coded.csv', delimiter = ',', impute = True, key = 'ID')

2020-03-19 12:12:21 Hail: INFO: Reading table to impute column types
2020-03-19 12:26:49 Hail: INFO: Finished type imputation
  Loading column 'ID' as type 'str' (imputed)
  Loading column 'family' as type 'str' (imputed)
  Loading column 'sex' as type 'str' (imputed)
  Loading column 'kinship' as type 'str' (imputed)
  Loading column 'disease' as type 'str' (imputed)
  Loading column 'phenotype' as type 'str' (imputed)
  Loading column 'add_pheno' as type 'str' (imputed)
  Loading column 'heavy_tics' as type 'str' (imputed)


In [21]:
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

In [22]:
mt = mt.annotate_rows(cases_with_gts = hl.agg.group_by(mt.phenotypes.family, hl.agg.count_where(mt.phenotypes.disease == 'YES')),
                      variants_with_gts = hl.agg.group_by((mt.phenotypes.family, mt.phenotypes.disease), hl.agg.count_where(mt.GT.is_non_ref())),
                      carriers_in_families = hl.agg.group_by(mt.s, hl.agg.count_where(mt.GT.is_non_ref())),
                      variants_in_sporadic_cases = hl.agg.group_by((mt.phenotypes.family, mt.phenotypes.disease), hl.agg.counter(mt.GT.is_non_ref()))
                     )
                     

In [23]:
mt = mt.annotate_rows(sporadic_cases = hl.str(mt.variants_in_sporadic_cases[('.','YES')]))

In [24]:
mt = mt.filter_rows((mt.gnomad_v3.v3_nfe.AF < 0.01))
mt = mt.filter_cols((mt.phenotypes.family != '.') & (hl.is_defined(mt.phenotypes.family)))
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [25]:
mt = mt.annotate_rows(variants_in_familial = hl.agg.group_by((mt.phenotypes.disease), hl.agg.counter(mt.GT.is_non_ref())))

In [26]:
mt = mt.annotate_rows(variants_in_familial_controls = hl.str(mt.variants_in_familial['NO']),
                      variants_in_familial_cases = hl.str(mt.variants_in_familial['YES']))

In [182]:
mt.variants_in_familial_cases.show()

locus,alleles,variants_in_familial_cases
locus<GRCh38>,array<str>,str
chr1:13801,"[""G"",""A""]","""[{""key"":false,""value"":66}]"""
chr1:13824,"[""G"",""A""]","""[{""key"":false,""value"":64},{""key"":true,""value"":2}]"""
chr1:13835,"[""G"",""C""]","""[{""key"":false,""value"":64},{""key"":true,""value"":2}]"""
chr1:15189,"[""C"",""T""]","""[{""key"":false,""value"":63},{""key"":true,""value"":2},{""key"":null,""value"":1}]"""
chr1:15259,"[""G"",""A""]","""[{""key"":false,""value"":62},{""key"":true,""value"":3},{""key"":null,""value"":1}]"""
chr1:15484,"[""G"",""T""]","""[{""key"":false,""value"":63},{""key"":true,""value"":3}]"""
chr1:15644,"[""G"",""A""]","""[{""key"":false,""value"":65},{""key"":true,""value"":1}]"""
chr1:15770,"[""G"",""A""]","""[{""key"":false,""value"":65},{""key"":true,""value"":1}]"""
chr1:15774,"[""G"",""A""]","""[{""key"":false,""value"":65},{""key"":true,""value"":1}]"""


In [183]:
mt.variants_in_familial_controls.show()

locus,alleles,variants_in_familial_controls
locus<GRCh38>,array<str>,str
chr1:13801,"[""G"",""A""]","""[{""key"":false,""value"":40},{""key"":true,""value"":2},{""key"":null,""value"":1}]"""
chr1:13824,"[""G"",""A""]","""[{""key"":false,""value"":43}]"""
chr1:13835,"[""G"",""C""]","""[{""key"":false,""value"":43}]"""
chr1:15189,"[""C"",""T""]","""[{""key"":false,""value"":42},{""key"":true,""value"":1}]"""
chr1:15259,"[""G"",""A""]","""[{""key"":false,""value"":41},{""key"":true,""value"":2}]"""
chr1:15484,"[""G"",""T""]","""[{""key"":false,""value"":38},{""key"":true,""value"":5}]"""
chr1:15644,"[""G"",""A""]","""[{""key"":false,""value"":42},{""key"":true,""value"":1}]"""
chr1:15770,"[""G"",""A""]","""[{""key"":false,""value"":40},{""key"":true,""value"":3}]"""
chr1:15774,"[""G"",""A""]","""[{""key"":false,""value"":43}]"""


In [30]:
mt.write('/hail/large_vcf_analysis/families.mt')

2020-03-19 13:43:23 Hail: INFO: wrote matrix table with 1588489 rows and 110 columns in 603 partitions to /hail/large_vcf_analysis/families.mt


In [31]:
mt = hl.read_matrix_table('/hail/large_vcf_analysis/families.mt')

In [32]:
fams = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'J', 'R', 'S', 'T'] # T family needs to be 

In [33]:
mtxs = []

# 'A' - sprawdź pierwsze dwie osoby z tej rodziny - dodaj, że mają tiki!!!!!

A = mt.filter_cols(mt.phenotypes.family == 'A')
A = A.filter_rows((hl.agg.any(A.GT.is_non_ref())) &
                    ((A.variants_with_gts[('A','YES')]) == (A.cases_with_gts['A'])) &
                    (A.carriers_in_families['S_7213'] == 0))


mtxs.append(A)


B = mt.filter_cols(mt.phenotypes.family == 'B')
B = B.filter_rows((hl.agg.any(B.GT.is_non_ref())) &
                    ((B.variants_with_gts[('B','YES')]) == (B.cases_with_gts['B'])) &
                    (B.carriers_in_families['S_7227'] == 0) &
                    (B.carriers_in_families['S_7255'] == 1))

mtxs.append(B)


C = mt.filter_cols(mt.phenotypes.family == 'C')
C = C.filter_rows((hl.agg.any(C.GT.is_non_ref())) &
                    ((C.variants_with_gts[('C','YES')]) == (C.cases_with_gts['C'])) &
                    (C.carriers_in_families['S_7241'] == 0))

mtxs.append(C)


D = mt.filter_cols(mt.phenotypes.family == 'D')
D = D.filter_rows((hl.agg.any(D.GT.is_non_ref())) &
                    ((D.variants_with_gts[('D','YES')]) == (D.cases_with_gts['D'])) &
                    (D.carriers_in_families['S_7246'] == 0) &
                    (D.carriers_in_families['S_7247'] == 1))

mtxs.append(D)


E = mt.filter_cols(mt.phenotypes.family == 'E')
E = E.filter_rows((hl.agg.any(E.GT.is_non_ref())) &
                    ((E.variants_with_gts[('E','YES')]) == (E.cases_with_gts['E'])) &
                    (E.carriers_in_families['S_7252'] == 1) &
                    (E.carriers_in_families['S_7254'] == 0))
mtxs.append(E)


F = mt.filter_cols(mt.phenotypes.family == 'F')
F = F.filter_rows((hl.agg.any(F.GT.is_non_ref())) &
                          ((F.variants_with_gts[('F','YES')]) == (F.cases_with_gts['F'])) &
                          (F.carriers_in_families['WGS_147c'] == 0))

mtxs.append(F)

H = mt.filter_cols(mt.phenotypes.family == 'H')
H = H.filter_rows((hl.agg.any(H.GT.is_non_ref())) &
                          ((H.variants_with_gts[('H','YES')]) == (H.cases_with_gts['H'])) &
                          (H.carriers_in_families['S_7274'] == 0))

mtxs.append(H)


I = mt.filter_cols(mt.phenotypes.family == 'I')
I = I.filter_rows((hl.agg.any(I.GT.is_non_ref())) &
                    ((I.variants_with_gts[('I','YES')]) == (I.cases_with_gts['I'])) &
                    (I.carriers_in_families['S_7291'] == 1))

mtxs.append(I)

J = mt.filter_cols(mt.phenotypes.family == 'J')
J = J.filter_rows((hl.agg.any(J.GT.is_non_ref())) &
                    ((J.variants_with_gts[('J','YES')]) == (J.cases_with_gts['J'])) &
                    (J.carriers_in_families['S_7307'] == 0))

mtxs.append(J)


R = mt.filter_cols(mt.phenotypes.family == 'R')
R = R.filter_rows((hl.agg.any(R.GT.is_non_ref())) &
                    ((R.variants_with_gts[('R','YES')]) == (R.cases_with_gts['R'])) &
                    (R.carriers_in_families['WGS_180b'] == 0))
mtxs.append(R)


S = mt.filter_cols(mt.phenotypes.family == 'S')
S = S.filter_rows((hl.agg.any(S.GT.is_non_ref())) &
                    ((S.variants_with_gts[('S','YES')]) == (S.cases_with_gts['S'])) &
                    (S.carriers_in_families['WGS_185c'] == 0) &
                    (S.carriers_in_families['WGS_6819'] == 0))
mtxs.append(S)



T = mt.filter_cols(mt.phenotypes.family == 'T')
T = T.filter_rows((hl.agg.any(T.GT.is_non_ref())) &
                    (T.carriers_in_families['WGS_6812'] == 0) &
                    (T.carriers_in_families['WGS_6814'] == 0) &
                    (T.carriers_in_families['WGS_D197a'] == 1) &
                    (T.carriers_in_families['WGS_D197b'] == 0) &
                    (T.carriers_in_families['WGS_D197c'] == 1) &
                    (T.carriers_in_families['WGS_D6811'] == 1) &
                    (T.carriers_in_families['WGS_D6813'] == 0) &
                    (T.carriers_in_families['WGS_D6815'] == 0) &
                    (T.carriers_in_families['WGS_D6816'] == 1))

    
mtxs.append(T)

In [34]:
tbs = []

colss = []

for mtx in mtxs:
    tb = mtx.select_cols(mtx.phenotypes.family, mtx.phenotypes.disease, mtx.phenotypes.sex, mtx.phenotypes.kinship, mtx.phenotypes.add_pheno,
                        mtx.phenotypes.heavy_tics)
    tb = tb.select_entries(tb.GT, tb.GQ)
    tb = tb.select_rows(tb.rsid, tb.gnomad_v3.v3_nfe.AF, tb.gnomad_v3.v3_nfe.homozygote_count,
                        tb.gnomad_v3['most_severe_consequence'], tb.cadd,
                        tb.nearest_genes_20kb, tb.sporadic_cases, tb.variants_in_familial_controls, tb.variants_in_familial_cases)
    cols = tb.cols()
    colss.append(cols.to_pandas())
    
    tb = tb.make_table()
    tbs.append(tb.to_pandas())

2020-03-19 13:43:24 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2020-03-19 13:43:24 Hail: INFO: Coerced sorted dataset
2020-03-19 13:43:42 Hail: INFO: Coerced sorted dataset
2020-03-19 13:43:55 Hail: INFO: Coerced sorted dataset
2020-03-19 13:44:08 Hail: INFO: Coerced sorted dataset
2020-03-19 13:44:21 Hail: INFO: Coerced sorted dataset
2020-03-19 13:44:33 Hail: INFO: Coerced sorted dataset
2020-03-19 13:44:45 Hail: INFO: Coerced sorted dataset
2020-03-19 13:44:58 Hail: INFO: Coerced sorted dataset
2020-03-19 13:45:11 Hail: INFO: Coerced sorted dataset
2020-03-19 13:45:25 Hail: INFO: Coerced sorted dataset
2020-03-19 13:45:38 Hail: INFO: Coerced sorted dataset
2020-03-19 13:45:51 Hail: INFO: Coerced sorted dataset


In [35]:
for col, tb in zip(colss, tbs):
    name = col.iloc[0]['family']

    col_name = 'cols_'+str(name)   
    row_name = 'rows_'+str(name)

    col.to_pickle('./py_objects/'+col_name)
    tb.to_pickle('./py_objects/'+row_name)

In [36]:
cols = !ls ./py_objects/cols*
rows = !ls ./py_objects/rows*

In [184]:
info = []
tables = []

In [185]:
for col, row in zip(cols, rows):
    info.append(pd.read_pickle(col))
    tables.append(pd.read_pickle(row))

results = []

In [186]:
for idx, table in enumerate(tables):
    
    a = list(table.columns)

    b = [x for x in a if 'phased' not in x]
    b = [x for x in b if 'GQ' not in x]
    
    table = table[b]
    table = table.astype(str)
    
    tables[idx] = table
    
    e = pd.DataFrame(table.columns).transpose()
    e.columns = table.columns
    f = pd.concat([e, table], axis=0)

    g = info[idx].transpose()
    h = pd.DataFrame(g.index)
    h.index = g.index
    i = pd.concat([h, g], axis = 1)
    
    i.columns = range(0,len(i.columns))
    i = i.reindex(columns = ['a']*11 + list(range(0,len(i.columns))))
    
    i.columns = f.columns
    i.index = range(0,len(i.index))

    f.index = range(len(i.index)+1,len(i.index)+len(f.index)+1)

    j = pd.concat([i, f], axis = 0)
    j.columns = range(0, len(j.columns))
    
    for z in [9,10,11]:
        j[z].replace(regex=True,inplace=True,to_replace={'\{','\}','\[','\"','\]','key:','value:','value'},value='')
    
        j[z].replace(regex=True,inplace=True,to_replace='false,',value='refs:')
        j[z].replace(regex=True,inplace=True,to_replace='true,',value='non_refs:')
        j[z].replace(regex=True,inplace=True,to_replace='::',value=':')
   
        j[z].replace(regex=True,inplace=True,to_replace='key:refs:',value='')
    
        j[z].replace(regex=True,inplace=True,to_replace='key:non_refs:',value='')
    
        j[z].replace(regex=True,inplace=True,to_replace='key:null',value='')
   
        j[z].replace(regex=True,inplace=True,to_replace=',,',value=',')
    
    k = j[9].str.split(',', expand=True)
    
    l = j[10].str.split(',', expand=True)
    
    m = j[11].str.split(',', expand=True)
    
    
    n = pd.concat([j[range(0,9)], k, l, m, j[range(12,len(j.columns))]], axis=1)
    
    
    n.columns = range(0, len(n.columns))
    
    n.iloc[7,9] = 'sporadic_cases_refs'
    n.iloc[7,10] = 'sporadic_cases_non_refs'
    n.iloc[7,11] = 'sporadic_cases_not_called'
    
    n.iloc[7,12] = 'families_controls_refs'
    n.iloc[7,13] = 'families_controls_non_refs'
    n.iloc[7,14] = 'families_controls_not_called'
    
    n.iloc[7,15] = 'families_cases_refs'
    n.iloc[7,16] = 'families_cases_non_refs'
    n.iloc[7,17] = 'families_cases_not_called'
    

    results.append(n)

In [187]:
with pd.ExcelWriter('families.xlsx') as writer:
    for idx, res in enumerate(results):
        res.to_excel(writer, sheet_name=str(res.iloc[1][18]), header=False, index=False)